## Drowsiness detection(from Pyimagesearch with some modifications)

In [1]:
##importing the dependencies
import cv2
import cvlib
import dlib
import numpy as np
from threading import Thread
import playsound
from scipy.spatial import distance as dist
import time


Using TensorFlow backend.


In [2]:
##helper functions
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 30
 
# initialize the frame counter as well as a boolean used to
# indicate if the alarm is going off
COUNTER = 0
ALARM_ON = False

def sound_alarm(path):
	# play an alarm sound
    playsound.playsound(path)

def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])

	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])

	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)

	# return the eye aspect ratio
	return ear


def shape_to_np(dlib_shape,dtype=int):
    coordinates = np.zeros((dlib_shape.num_parts,2),dtype)
    
    for i in range(0,dlib_shape.num_parts):
        coordinates[i] = (dlib_shape.part(i).x,dlib_shape.part(i).y)
        
    return coordinates

In [4]:
p = "models/shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(p)
right_eye_points = list(range(36,42))
left_eye_points = list(range(42,48))
path = "alarm.wav"
vs = cv2.VideoCapture(0)

while True:
    
    ret, frame = vs.read()    
    rects,confidence = cvlib.detect_face(frame,0.7)
    
    for rect in rects:
        left = rect[0]
        top = rect[1]
        right = rect[2]
        bottom = rect[3]

        shape = predictor(frame,dlib.rectangle(left,top,right,bottom))
        np_shape = shape_to_np(shape)
        right_eye = np_shape[right_eye_points]
        left_eye = np_shape[left_eye_points]
        rightEAR = eye_aspect_ratio(right_eye)
        leftEAR = eye_aspect_ratio(left_eye)
        
        ear = (leftEAR+rightEAR) / 2.0
        # compute the convex hull for the left and right eye, then
        # visualize each of the eyes
        leftEyeHull = cv2.convexHull(left_eye)
        rightEyeHull = cv2.convexHull(right_eye)
        cv2.drawContours(frame,[leftEyeHull],-1,(0,255,0),2)
        cv2.drawContours(frame,[rightEyeHull],-1,(0,255,0),2)
        
        
        if ear < EYE_AR_THRESH:
            COUNTER += 1
            
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                if not ALARM_ON:
                    ALARM_ON = True
                    
                    t = Thread(target=sound_alarm,args=(path,))
                    t.daemon  = True
                    t.start()
                cv2.putText(frame,'DROWSINESS ALERT!',(left,top),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
                    
        else:
            COUNTER = 0
            ALARM_ON = False
        cv2.putText(frame,f'EAR :{ear:.3f}',(right,top),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2.imshow('Frame',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
vs.release()
cv2.destroyAllWindows()